<a href="https://colab.research.google.com/github/jmfdesouza/PUC_DataSciense/blob/main/PUC_MVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np
from datetime import *
pd.set_option("mode.chained_assignment", None)

data_base = pd.to_datetime(date(2024, 9, 27).strftime("%m/%d/%Y"))
nsua = 13
tx_juros = 4.43
v = 1/(1+(tx_juros/100))
fat_sup = 1.0
nsua = 13
tx_adm = 4.0
tx_inf = 3.5
fcb = 0.9830029234686
ur = 4000.00
teto_efpc = 6340.83
teto_inss=7810.68
ind_cor = 1.03125151949196

base_dados_ativos = pd.read_excel("https://github.com/jmfdesouza/PUC_DataSciense/blob/main/PUC%20MVP%20-%20T%C3%A1buas%20Biom%C3%A9tricas.xlsx", sheet_name="base_dados_ativos")
# # Convertendo as datas da base de dados para formato Python
base_dados_ativos.loc[:, "DATA_NASC"] = pd.to_datetime(base_dados_ativos.iloc[1:, 1], format="%d/%m/%y")
base_dados_ativos.loc[:, "DATA_ADM"] = pd.to_datetime(base_dados_ativos.iloc[1:, 3], format="%d/%m/%y")
base_dados_ativos.loc[:, "DATA_ADES"] = pd.to_datetime(base_dados_ativos.iloc[1:, 2], format="%d/%m/%y")
data_reestrut = pd.to_datetime(date(2020, 5, 5).strftime("%m/%d/%Y"))
base_dados_ativos.loc[:, "SALARIO_ATIVO"] = (base_dados_ativos.iloc[1:, 4].astype(float)*(13/12)*1.0078).round(2)
base_dados_ativos.loc[:, "IDADE_ATUAL"] = data_base.year - base_dados_ativos["DATA_NASC"].dt.year
base_dados_ativos.loc[:, "IDADE_05/05/2020"]=data_reestrut.year-base_dados_ativos["DATA_NASC"].dt.year
base_dados_ativos.loc[:, "TVP_REAL"] = (base_dados_ativos.iloc[1:, 13]/12).round()
base_dados_ativos.loc[:, "TVP_AJUST"] = np.where((base_dados_ativos["TVP_REAL"]/12- base_dados_ativos["TVP_REAL"])*12>5, base_dados_ativos["TVP_REAL"], base_dados_ativos["TVP_REAL"]-1)
base_dados_ativos.loc[:, "TEMPO_CONT_INSS"] = np.where(base_dados_ativos.iloc[:, 8]=="M", 35, 30)
base_dados_ativos.loc[:, "RESID_AP_CONT"] = np.maximum(base_dados_ativos["TEMPO_CONT_INSS"] - base_dados_ativos["TVP_AJUST"], 0)
base_dados_ativos.loc[:, "IDADE_INSS"] = np.where(base_dados_ativos.iloc[:, 8]=="M", 65, 60)
base_dados_ativos.loc[:, "RESID_AP_IDADE"] = np.maximum(base_dados_ativos["IDADE_INSS"] - base_dados_ativos["IDADE_05/05/2020"], 0)
base_dados_ativos.loc[:, "DIR_ADQ"] = np.where(pd.to_datetime(base_dados_ativos.iloc[:, 15]) <= data_reestrut, "S", "N")
base_dados_ativos["DIR_ADQ"] = np.where(np.minimum(base_dados_ativos["RESID_AP_CONT"], base_dados_ativos["RESID_AP_IDADE"])<=0, "S", "N")
base_dados_ativos.loc[:,"IDADE_AP_INSS"]=np.where(base_dados_ativos.iloc[:, 14].fillna("N")=="S", base_dados_ativos["IDADE_ATUAL"], base_dados_ativos["IDADE_ATUAL"]+np.minimum(np.maximum(np.where(base_dados_ativos.iloc[:, 8]=="M", 35, 30)- base_dados_ativos["TVP_REAL"], 0), np.maximum(0, np.where(base_dados_ativos.iloc[:, 8]=="M", 65, 60)- base_dados_ativos['IDADE_ATUAL'])))
base_dados_ativos.loc[:,"IDADE_APOSENT"]=np.where(base_dados_ativos["DIR_ADQ"]=="N", np.maximum(55, base_dados_ativos["IDADE_ATUAL"], base_dados_ativos["IDADE_05/05/2020"]), np.maximum(np.maximum(55,  base_dados_ativos["IDADE_AP_INSS"]), base_dados_ativos["IDADE_ATUAL"]))
data_14_04_1982 = pd.to_datetime(date(1982, 4, 14).strftime("%m/%d/%Y"))
base_dados_ativos.loc[:, "LSP"] = np.where(base_dados_ativos["DATA_ADES"]<data_14_04_1982, 28160.85, 3*teto_inss*ind_cor).round(2)
base_dados_ativos.loc[:, "SALARIO_ATUARIAL"] = np.minimum(base_dados_ativos["SALARIO_ATIVO"]*1.0078**(np.maximum(0, np.minimum(100, base_dados_ativos["IDADE_APOSENT"])-base_dados_ativos["IDADE_ATUAL"])), base_dados_ativos["LSP"])
base_dados_ativos.loc[:, "INSS_REAL"]=np.where(base_dados_ativos.iloc[:, 14]=="S", base_dados_ativos.iloc[:, 16], "-")
tabua_ibge = pd.read_excel("https://github.com/jmfdesouza/PUC_DataSciense/blob/main/PUC%20MVP%20-%20T%C3%A1buas%20Biom%C3%A9tricas.xlsx", sheet_name="tabua_IBGE")
base_dados_ativos=pd.merge(base_dados_ativos, tabua_ibge[["Unnamed: 0", "Unnamed: 6"]], left_on="IDADE_AP_INSS", right_on="Unnamed: 0", how="left")
base_dados_ativos.rename(columns={"Unnamed: 6": "EXPECT_AP_INSS"}, inplace=True)
base_dados_ativos.loc[:, "N_PONTOS"]=base_dados_ativos["IDADE_ATUAL"]+base_dados_ativos["TVP_REAL"]
base_dados_ativos.loc[:, "ANO_APOSENT_INSS"]=np.where(base_dados_ativos["N_PONTOS"]==96, 2020, np.where(base_dados_ativos["N_PONTOS"]==97, 2021, np.where(base_dados_ativos["N_PONTOS"]==98, 2023, np.where(base_dados_ativos["N_PONTOS"]==99, 2025, 2027))))
base_dados_ativos.loc[:, "RESID_AP_PONTOS"]=base_dados_ativos["ANO_APOSENT_INSS"]-data_base.year
base_dados_ativos.loc[:,"TIPO_APOSENT"]=np.where(base_dados_ativos["RESID_AP_IDADE"]<=np.minimum(base_dados_ativos["RESID_AP_CONT"], base_dados_ativos["RESID_AP_PONTOS"]), "AP_IDADE", np.where(base_dados_ativos["RESID_AP_PONTOS"]<=np.minimum(base_dados_ativos["RESID_AP_CONT"], base_dados_ativos["RESID_AP_IDADE"], base_dados_ativos["RESID_AP_PONTOS"]), "AP_PONTOS", "AP_CONT"))
base_dados_ativos.loc[:,"FATOR_PREV"]=np.where(base_dados_ativos["TIPO_APOSENT"]==("AP_PONTOS" or "AP_IDADE"), np.maximum(1, 0.31*(np.where(base_dados_ativos.iloc[:, 8]=="F", 5, 0)+(base_dados_ativos["IDADE_AP_INSS"]-(base_dados_ativos["IDADE_ATUAL"]-base_dados_ativos["TVP_REAL"])))/base_dados_ativos["EXPECT_AP_INSS"]*(1+(base_dados_ativos["IDADE_AP_INSS"]+0.31*(np.where(base_dados_ativos.iloc[:, 8]=="F", 5, 0)+(base_dados_ativos["IDADE_AP_INSS"]-(base_dados_ativos["IDADE_ATUAL"]-base_dados_ativos["TVP_REAL"]))))/100)), 0.31*(np.where(base_dados_ativos.iloc[:, 8]=="F", 5, 0)+( base_dados_ativos["IDADE_AP_INSS"]-(base_dados_ativos["IDADE_ATUAL"]-base_dados_ativos["TVP_REAL"])))/base_dados_ativos["EXPECT_AP_INSS"]*(1+( base_dados_ativos["IDADE_AP_INSS"]+0.31*(np.where(base_dados_ativos.iloc[:, 8]=="F", 5, 0)+(base_dados_ativos["IDADE_AP_INSS"]-(base_dados_ativos["IDADE_ATUAL"]-base_dados_ativos["TVP_REAL"]))))/100))
base_dados_ativos.loc[:,"PHI"]=np.where(base_dados_ativos.iloc[:, 8]=="M", np.where(base_dados_ativos["TVP_REAL"]<=34, 0.7+0.05*np.minimum(5, base_dados_ativos["TVP_REAL"]-30),1), np.where(base_dados_ativos["TVP_REAL"]<=29, 0.7+0.05*np.minimum(5, base_dados_ativos["TVP_REAL"]-25), 1))
base_dados_ativos.loc[:, "SB_AP"]= 120.000/((data_base.year + base_dados_ativos["IDADE_AP_INSS"]-base_dados_ativos["IDADE_ATUAL"])-1994-(0.2*((data_base.year + base_dados_ativos["IDADE_AP_INSS"]-base_dados_ativos["IDADE_ATUAL"])-1994)))
base_dados_ativos.loc[:, "INSS_ATUARIAL"]=np.where(base_dados_ativos["DIR_ADQ"]=="N", ur*ind_cor, np.where(base_dados_ativos.iloc[:, 14]=="S", base_dados_ativos.iloc[:, 16], np.minimum(teto_efpc, base_dados_ativos["FATOR_PREV"]*base_dados_ativos["SB_AP"])*base_dados_ativos["PHI"]))
base_dados_ativos.loc[:, "KA"]=np.minimum(1, (np.where(base_dados_ativos["DIR_ADQ"]=="N", base_dados_ativos["IDADE_APOSENT"], base_dados_ativos["IDADE_AP_INSS"])-( base_dados_ativos["IDADE_ATUAL"]-base_dados_ativos["TVP_REAL"])+np.where(base_dados_ativos.iloc[:, 8]=="M", 0, 5))/35)*np.minimum(1, (base_dados_ativos["IDADE_APOSENT"]-base_dados_ativos["IDADE_ATUAL"]+(data_base.year-base_dados_ativos["DATA_ADM"].dt.year))/10)
base_dados_ativos.loc[:, "SAP"]=np.minimum(np.maximum(0, (base_dados_ativos["SALARIO_ATUARIAL"]-base_dados_ativos["INSS_ATUARIAL"])*base_dados_ativos["KA"], 0.1*np.minimum(teto_efpc, base_dados_ativos["SALARIO_ATUARIAL"])), base_dados_ativos["LSP"]-base_dados_ativos["INSS_ATUARIAL"]).astype(float)
base_dados_ativos.loc[:,"T_ASS"]=(np.where(base_dados_ativos.iloc[:,10]=="S", 1.96/100, 1.45/100)* base_dados_ativos["SAP"]*fat_sup+np.where(base_dados_ativos.iloc[:,10]=="S", 2.1/100, 1.55/100)*np.maximum(0, base_dados_ativos["SAP"]*fat_sup-teto_efpc/2)+np.where(base_dados_ativos.iloc[:,10]=="S", 10.84/100, 8/100)*np.maximum(0, base_dados_ativos["SAP"]*fat_sup-teto_efpc))/(base_dados_ativos["SAP"]*fat_sup)

tabua_lx = pd.read_excel("https://github.com/jmfdesouza/PUC_DataSciense/blob/main/PUC%20MVP%20-%20T%C3%A1buas%20Biom%C3%A9tricas.xlsx", sheet_name="tabua_lx")
base_dados_ativos=pd.merge(base_dados_ativos, tabua_lx[["Parâmetros Financeiros", "Unnamed: 1"]], left_on="IDADE_ATUAL", right_on="Parâmetros Financeiros", how="left")
base_dados_ativos.rename(columns={"Unnamed: 1_x": "LX"}, inplace=True)

base_dados_ativos=pd.merge(base_dados_ativos, tabua_lx[["Parâmetros Financeiros", "Unnamed: 1"]], left_on="IDADE_APOSENT", right_on="Parâmetros Financeiros", how="left")
base_dados_ativos.rename(columns={"Unnamed: 1_y": "LR"}, inplace=True)
base_dados_ativos.loc[:, "RESID_AP_EFPC"] = base_dados_ativos["IDADE_APOSENT"]-base_dados_ativos["IDADE_ATUAL"]

def calc_fluxo_ativos():
  result_ativos = pd.DataFrame()
  for n in tabua_lx.iloc[6:122, 0]:
    prob_ativos=pd.merge(base_dados_ativos["IDADE_APOSENT"], tabua_lx[["Parâmetros Financeiros", "Unnamed: 1"]], left_on="IDADE_APOSENT", right_on="Parâmetros Financeiros", how="left")
    anuidade = nsua * base_dados_ativos["SAP"] * fat_sup * (1-base_dados_ativos["T_ASS"]*(1-tx_adm)) * fcb * (((prob_ativos["Unnamed: 1"])/base_dados_ativos["LR"]) * v ** n) * ((base_dados_ativos["LR"]/base_dados_ativos["Unnamed: 1_x"]) * v ** base_dados_ativos["RESID_AP_EFPC"])
    result_ativos[data_base.year+1+n] = anuidade
    base_dados_ativos["IDADE_APOSENT"] += 1
  return result_ativos.sum()

def calc_consolidado_ativos():
    return calc_fluxo_ativos().sum()

display(calc_fluxo_ativos().head(62))


ValueError: Excel file format cannot be determined, you must specify an engine manually.